In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime,timedelta

import bokeh.palettes as bp
from bokeh.palettes import inferno
from bokeh.plotting import figure,curdoc
from bokeh.transform import linear_cmap
from bokeh.layouts import column, row, widgetbox #last from web
from bokeh.palettes import brewer #from web
from bokeh.io import show #from web
from bokeh.models import (CDSView, 
						HoverTool,ColorBar,
						GeoJSONDataSource, 
						Patches,
						RadioButtonGroup,
						DateSlider,
						Button,
						ColumnDataSource, CustomJS, CustomJSFilter, LinearColorMapper, Slider) #from web


# ====================================================================
# Goal: Visualize demographics and daily new cases statistics in Switzerland

# The visualization output contains 2 parts: 

# Part 1: add a map view and the color encodes the Density and BedsPerCapita, 
# with a RadioButtonGroup which controls which aspect to show. 


# Part 2: add a circle in each canton on the map, size of circle represents the current daily new cases per capita,
# add a timeline slider, add callback function controlling the slider and changing the size of the circle on the map 
# when dragging the time slider.
# Additionally, a "Play" button can animate the slider as well as circles on the map.  

# Reference link
# https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

# ====================================================================



### + Task 1: Data Preprocessing

## T1.1 Read and filter data 
# Four data sources:
# Demographics.csv: the statistics data about population density and hospital beds per capita in each canton
# covid_19_cases_switzerland_standard_format.csv: the location(longitude, latitude) of the capital city in each canton
# covid19_cases_switzerland_openzh-phase2.csv: same as in ex2, daily new cases in each canton
# gadm36_CHE_1.shp: the shape file contains geometry data of swiss cantons, and is provided in the "data" folder. 
# Please do not submit any data file with your solution, and you can asssume your solution is at the same directory as data 


demo_url = 'https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/demographics.csv?raw=true'
local_url = 'https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/covid_19_cases_switzerland_standard_format.csv?raw=true'
case_url = 'https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/covid19_cases_switzerland_openzh-phase2.csv?raw=true'
shape_dir = 'data/gadm36_CHE_1.shp'


# Read from demo_url into a dataframe using pandas
demo_raw = pd.read_csv(demo_url)
# print("demo_raw")
# print(demo_raw.head())


# Read from local_url into a dataframe using pandas
local_raw = pd.read_csv(local_url)
# print(local_raw.head())

# Extract unique 'abbreviation_canton','lat','long' combinations from local_raw
canton_point = local_raw[["abbreviation_canton", "lat", "long"]] 
canton_point = canton_point.drop_duplicates()
# print(canton_point.head(30))


# Read from case_url into a dataframe using pandas
case_raw = pd.read_csv(case_url) 
# print(case_raw.head())
# Create a date list from case_raw and convert to datatime form
dates = list(pd.to_datetime(case_raw['Date']))
# print(dates_raw)
# print(dates)


# Read shape file from shape_dir unsing geopandas
shape_raw = gpd.read_file(shape_dir)
# shape_raw.head()
# Extract canton name abbreviations from the attribute 'HASC_1', e.g. CH.AG --> AG, CH.ZH --> ZH
# And save into a new column named 'Canton' 
shape_raw['Canton'] = shape_raw['HASC_1'].replace(regex='CH.', value='')
# print(shape_raw.head())
canton_poly = shape_raw[['geometry','Canton']]
# print("canton_poly")
# print(canton_poly.head())





## T1.2 Merge data and build a GeoJSONDataSource

# Merge canton_poly with demo_raw on attribute name 'Canton' into dataframe merged,
# then merge the result with canton_point on 'Canton' and 'abbreviation_canton' respectively


# Potential issue: 
# https://stackoverflow.com/questions/57045479/is-there-a-way-to-fix-maximum-recursion-level-in-python-3
merged = canton_poly.merge(demo_raw, how = "left", on = "Canton")
# print("First merge")
# print(merged.head())

merged = merged.merge(canton_point, left_on = "Canton", right_on = "abbreviation_canton")
merged = merged.drop(columns = ["abbreviation_canton"])
# print("Second merge")
# print(merged.head())
# print(list(merged["Canton"]))


# For each date, extract a list of daily new cases per capita from all cantons(e.g. 'AG_diff_pc', 'AI_diff_pc', etc.), and add as a new column in merged
# For instance, in the column['2020-10-31'], there are: [0.0005411327220155498, nan, nan, 0.000496300306803826, ...]
dates_raw = case_raw[["Date"]].copy()
for index, row in merged.iterrows():
	canton = row["Canton"]
	column_name = canton + "_diff_pc"
	dates_raw[column_name] = case_raw[[column_name]]
dates_raw = dates_raw.set_index("Date")
# print(dates_raw.loc[dates_raw["Date"] == '2020-10-31'])
# print(dates_raw)

for i, row in dates_raw.iterrows():
	merged[i] = list(row)
# print("Third merge")
# print(merged)
# print(merged['2020-10-31'])

# Calculate circle sizes that are proportional to dnc per capita
# Set the latest dnc as default 
merged['dnc'] = merged.iloc[:,-1]
# merged['dnc'] = merged['2020-12-15']
merged['size'] = merged.iloc[:,-1]*1e5/5+10
merged_col = list(merged.columns)
print(merged_col[:12])
# print(merged['BedsPerCapita'].min())
# print(merged['lat'])
# print(merged['2020-12-15'])
# print(merged.iloc[:,-1])


#+ Build a GeoJSONDataSource from merged
geosource = GeoJSONDataSource(geojson = merged.to_json())



# Task 2: Data Visualization

#+ T2.1 Create linear color mappers for 2 attributes in demo_raw: population density, hospital beds per capita 
# Map their maximum values to the high, and mimimum to the low
color_palette = list(inferno(26))
color_palette = color_palette[::-1]
mappers = {} 
mappers['Density'] = LinearColorMapper(palette = color_palette, low = merged['Density'].min(), high = merged['Density'].max())
mappers['BedsPerCapita'] = LinearColorMapper(palette = color_palette, low = merged['BedsPerCapita'].min(), high = merged['BedsPerCapita'].max())


#+ T2.2 Draw a Switzerland Map on canton level

#+ Define a figure 
p1 = figure(title = 'Demographics in Switzerland', 
					 plot_height = 600 ,
					 plot_width = 950, 
					 toolbar_location = 'above',
					 tools = "pan, wheel_zoom, box_zoom, reset")

p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None

#+ Plot the map using patches, set the fill_color as mappers['Density']
cantons = p1.patches('xs','ys', source = geosource,
                   fill_color = {'field' : 'Density',
                                 'transform' : mappers['Density']}, 
                   line_width = 0.25, 
                   fill_alpha = 0.7)


#+ Create a colorbar with mappers['Density'] and add it to above figure
color_bar = ColorBar(color_mapper = mappers['Density'], location=(0,0), width=15, 
                     title = 'Density')
p1.add_layout(color_bar, 'right')


#+ Add a hovertool to display canton, density, bedspercapita and dnc 
hover = HoverTool(renderers = [cantons],
                  tooltips = [('Canton','@Canton'),
                              ('Population Density','@Density{0.}'),
							  ('Beds Per Capita', '@BedsPerCapita'),
							  ('Daily New Cases per Capita', '@dnc')])

p1.add_tools(hover)

#+ # T2.3 Add circles at the locations of capital cities for each canton, and the sizes are proportional to daily new cases per capita
# merged['lat'], merged['long']
source = ColumnDataSource(dict(
    lat = merged['lat'],
    long = merged['long'],
	size = merged['size']))
# sites = p1.circle(x = 'lat', y = 'long', source = source, color = 'red', 
#                  size = 30, alpha = 1)
sites = p1.circle(x = 'long', y = 'lat', source = source, color = 'red', 
                 size = 'size', alpha = 0.3)


##########################
##########################
# # T2.4 Create a radio button group with labels 'Density', and 'BedsPerCapita'
labels = ['Density','BedsPerCapita']
buttons = RadioButtonGroup(labels=labels, active=0)
for i,d in enumerate(labels):
	print (i, d)

# # Define a function to update color mapper used in both patches and colorbar 
def update_bar(new):
	for i,d in enumerate(labels):
		if i == new:
			color_bar.color_mapper = mappers[d]["transform"]
			color_bar.title = d
			cantons.glyph.fill_color = mappers[d]
			


buttons.on_click(update_bar)


# T2.5 Add a dateslider to control which per capita daily new cases information to display

# Define a dateslider using maximum and mimimum dates, set value to be the latest date
timeslider = DateSlider(title = "Date",
						start = dates[0], end = dates[-1])

# # Complete the callback function 
# # Hints: 
# # 	convert the timestamp value from the slider to datetime and format it in the form of '%Y-%m-%d'
# #	update columns 'size', 'dnc' with the column named '%Y-%m-%d' in merged
# #	update geosource with new merged

# def callback(attr,old,new):
# 	# Convert timestamp to datetime
# 	# https://stackoverflow.com/questions/9744775/how-to-convert-integer-timestamp-to-python-datetime
# 	date = ...
# 	i = date.strftime('%Y-%m-%d')

# 	merged.size = ...
# 	merged.dnc = ...
# 	geosource.geojson = ...


# # Circles change on mouse move
# timeslider.on_change(...)


# # T2.6 Add a play button to change slider value and update the map plot dynamically
# # https://stackoverflow.com/questions/46420606/python-bokeh-add-a-play-button-to-a-slider
# # https://stackoverflow.com/questions/441147/how-to-subtract-a-day-from-a-date

# # Update the slider value with one day before current date
# def animate_update_slider():
# 	# Extract date from slider's current value 
# 	date = ...
# 	# Subtract one day from date and do not exceed the allowed date range
# 	day = ...

# 	...

# 	timeslider.value = day

# # Define the callback function of button
# def animate():
# 	global callback_id
# 	if button.label == '► Play':
# 		button.label = '❚❚ Pause'
# 		callback_id = curdoc().add_periodic_callback(animate_update_slider, 500)
# 	else:
# 		button.label = '► Play'
# 		curdoc().remove_periodic_callback(callback_id)

# button = Button(label='► Play', width=80, height=40)
# button.on_click(animate)




# curdoc().add_root(column(p1,buttons, row(timeslider,button)))
curdoc().add_root(column(p1,buttons))

# linked_p = column(p1, buttons, timeslider)
# show(linked_p)
print("All done!")

['geometry', 'Canton', 'Population', 'SettlementAreaHa', 'SettlementAreaKm2', 'Density', 'O65', 'O65P', 'Beds', 'BedsPerCapita', 'lat', 'long']
0 Density
1 BedsPerCapita
All done!


['geometry', 'Canton', 'Population', 'SettlementAreaHa', 'SettlementAreaKm2', 'Density', 'O65', 'O65P', 'Beds', 'BedsPerCapita', 'lat', 'long']
0.001114896
0     47.449406
1     47.382710
2     47.328414
3     47.482779
4     47.558395
5     46.916667
6     46.718391
7     46.195602
8     47.040570
9     46.796756
10    47.350744
11    47.083333
12    46.995534
13    46.958050
14    46.898509
15    47.431620
16    47.713570
17    47.061787
18    47.206649
19    47.559930
20    46.009279
21    46.880422
22    46.209567
23    46.536900
24    47.157296
25    47.451542
Name: lat, dtype: float64
